# SPMT
## Import Libraries

In [ ]:
import pandas as pd
import numpy as np
from IPython.display import display
from scipy.spatial.distance import cdist
from scipy.optimize import linear_sum_assignment

## Student Intake Survey

Update settings to print more dataframe rows and columns at a time.

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

Read CSV and drop irrelevent columns. May or may not be necessary depending on data given.

In [ ]:
# read csv from Qualtrics into pandas dataframe, skip first 2 rows (does not include header, just question info)
df = pd.read_csv("qualtrics_file_name.csv", skiprows=[1,2])

# drop irrelevant columns
df = df.drop(columns=["StartDate", "EndDate", "Status", "IPAddress", "Progress", "Duration (in seconds)", "Finished", \
                "RecordedDate", "ResponseId", "RecipientEmail", "ExternalReference", "LocationLatitude", \
                "LocationLongitude", "DistributionChannel", "UserLanguage"])

# replace blank cells (no answer) with 0s
df = df.replace(np.nan, 0)

Display the first few responses to verify that our data cleaning has been successful.

In [ ]:
df.head()

## Student Intake Survey - Skills

Average out students' experience in diverse skills into a single score per category.

In [ ]:
df["Web_Score"] = (df["F1_1"] + df["WEB1_1"] + df["WEB2_1"] + df["WEB3_1"] + df["WEB4_1"] + df["WEB4_2"] + df["WEB4_3"] \
    + df["WEB4_4"]) / 8

df["Mobile_Score"] = (df["F1_1"] + df["MOB1_1"] + df["MOB1_2"] + df["MOB1_3"] + df["MOB1_4"]) / 5

df["Backend_Score"] = (df["CBI1_1"] + df["B1_1"] + df["B1_2"] + df["B1_3"] + df["B1_4"] + df["B1_5"] + df["B1_6"] \
    + df["B2_1"] + df["B2_2"] + df["B2_3"] + df["B2_4"] + df["B2_5"] + df["B3_1"]) / 13

df["DB_Score"] = (df["CBI1_1"] + df["DB1_1"] + df["DB1_2"] + df["DB1_3"] + df["DB1_4"] + df["DB1_5"]) / 6

df["Network_Score"] = (df["CBI1_1"] + df["NET1_1"]) / 2

df["ML_Score"] = (df["CBI1_1"] + df["ML1_1"] + df["ML2_1"] + df["ML3_1"] + df["ML3_2"] + df["ML3_3"] + df["ML4_1"] \
    + df["ML4_2"] + df["ML4_3"]) / 9

df["Data_Science_Score"] = (df["CBI1_1"] + df["DS1_1"] + df["DS1_2"] + df["DS1_3"] + df["DS1_4"] + df["DS1_5"] \
    + df["DS2_1"] + df["DS2_2"] + df["DS2_3"] + df["DS3_1"] + df["DS3_2"] + df["DS3_3"]) / 12



Check first few responses to ensure averaging is successful.

In [ ]:
df.head()

## Student Intake Survey - Interests

Creating scores for students' interest levels in each category

In [ ]:
df['web_interest'] = df.apply(lambda row: 1 if 'Web' in str(row['FSPEC']) or 'Not sure' in str(row['FSPEC']) else (0.5 if 'Maybe' in str(row['TPF']) else 0), axis=1)

In [ ]:
df['mobile_interest'] = df.apply(lambda row: 1 if 'Mobile' in str(row['FSPEC']) or 'Not sure' in str(row['FSPEC']) else (0.5 if 'Maybe' in str(row['TPF']) else 0), axis=1)

In [ ]:
df['backend_interest'] = df['TPB'].astype(str).apply(lambda x: 1 if 'Yes' in x else(0.5 if 'Maybe' in x else 0))

In [ ]:
df['db_interest'] = df.apply(lambda row: 1 if 'Databases' in str(row['ISPEC']) or 'Not sure' in str(row['ISPEC']) else (0.5 if 'Maybe' in str(row['TPI']) else 0), axis=1)

In [ ]:
df['network_interest'] = df.apply(lambda row: 1 if 'Network systems' in str(row['ISPEC']) or 'Not sure' in str(row['ISPEC']) else (0.5 if 'Maybe' in str(row['TPI']) else 0), axis=1)

In [ ]:
df['ml_interest'] = df.apply(lambda row: 1 if 'ML/AI' in str(row['ISPEC']) or 'Not sure' in str(row['ISPEC']) else (0.5 if 'Maybe' in str(row['TPI']) else 0), axis=1)

In [ ]:
df['ds_interest'] = df.apply(lambda row: 1 if 'Data science' in str(row['ISPEC']) or 'Not sure' in str(row['ISPEC']) else (0.5 if 'Maybe' in str(row['TPI']) else 0), axis=1)

In [ ]:
df.head()

Drop duplicates based on student email and keep the last occurrence (students were allowed to submit multiple times).

In [ ]:
df = df.drop_duplicates(subset=['Q50'],keep='last')
df.shape

Create a dataframe of student interests.

In [ ]:
std_interests = df[['Q50', 'web_interest', 'mobile_interest', 'backend_interest', 'db_interest', 'network_interest', 'ml_interest', 'ds_interest']]
std_interests.columns = ['netid', 'web', 'mobile', 'backend', 'db', 'network', 'ml', 'ds']
std_interests.head()

Create a dataframe of student interests in each project plus their preference on being a team lead.

In [ ]:
std_priorities = df[['Q50', 'priority1', 'priority2', 'priority3', 'priority4', 'priority5',
                     'priority6', 'priority7', 'priority8', 'priority9', 'priority10',
                     'priority11', 'priority12', 'priority13', 'priority14', 'team_lead_pref']]
std_priorities.columns = ['netid', 'priority1', 'priority2', 'priority3', 'priority4', 'priority5',
                     'priority6', 'priority7', 'priority8', 'priority9', 'priority10',
                     'priority11', 'priority12', 'priority13', 'priority14', 'team_lead_pref']

In [ ]:
priorities = {
    '1 - Highest Priority': 1,
    '2 - Average Priority': 2,
    '3 - Lowest Priority': 3
}

In [ ]:
std_priorities = std_priorities.replace(priorities)
std_priorities = std_priorities.fillna(3)

In [ ]:
std_priorities.head()

## Sponsor Projects

In [ ]:
# read csv of sponsor responses about project technical requirements; skip first 2 rows (does not include header, just question info)
spon = pd.read_csv("sponsor_csv.csv")

# # drop irrelevant columns
# spon = spon.drop(columns=["StartDate", "EndDate", "Status", "IPAddress", "Progress", "Duration (in seconds)", "Finished", \
#                 "RecordedDate", "ResponseId", "RecipientEmail", "ExternalReference", "LocationLatitude", \
#                 "LocationLongitude", "DistributionChannel", "UserLanguage"])

In [ ]:
spon[:2]

In [ ]:
spon_temp = spon[['contact_1', 'ai_imp', 'db_imp', 'networks_imp', 'web_imp', 'backend_imp', 'frontend_imp', 'mobile_imp', 'ds_imp', 'testing_imp', 'cloud_imp', 'cicd_imp', 'security_imp']]
spon_temp[:2]

#### Map values to numbers

In [ ]:
imps = {
    'Very important - major project component':1,
    'Moderately important - significant project component':.8,
    'Slightly important - small project component':.6,
    'Not at all important - not applicable to the project':0
}

In [ ]:
spon_temp = spon_temp.replace(imps)
spon_temp = spon_temp.fillna(0)

spon_temp[:2]

Creating scores for project technical needs.

In [ ]:
spon_needs = pd.DataFrame()

spon_needs['sponsor'] = spon_temp['contact_1']

spon_needs["web"] = (spon_temp['web_imp'] + spon_temp['testing_imp'] + spon_temp['cicd_imp']) / 3

spon_needs["mobile"] = (spon_temp['mobile_imp'] + spon_temp['testing_imp'] + spon_temp['cicd_imp']) / 3

spon_needs["backend"] = (spon_temp['testing_imp'] + spon_temp['cloud_imp'] + spon_temp['cicd_imp'] + spon_temp['security_imp']) / 4

spon_needs["db"] = spon_temp['db_imp']

spon_needs["network"] = spon_temp['networks_imp']

spon_needs["ml"] = spon_temp['ai_imp']

spon_needs["ds"] = spon_temp['ds_imp']

In [ ]:
spon_needs

## Setting up the multipliers based on interest level

In [ ]:
spon_needs['sponsor'].value_counts()

In [ ]:
for i in std_priorities.columns:
    print(std_priorities[i].value_counts())

In [ ]:
std_priorities[:2]

Expand each project to the total number of slots available in that project.

In [ ]:
prty_pt1 = std_priorities[['priority1', 'priority2', 'priority3', 'priority4', 'priority5', 'priority6', 'priority7']].values.repeat(5,axis=1)
prty_pt2 = std_priorities[['priority8', 'priority9', 'priority10', 'priority11']].values.repeat(6,axis=1)
prty_pt3 = std_priorities[['priority12', 'priority13', 'priority14']].values.repeat(5,axis=1)

prty_np = np.concatenate([prty_pt1,prty_pt2,prty_pt3], axis=1)

prty_df = pd.DataFrame(prty_np)

In [ ]:
prty_df.shape

In [ ]:
prty_df[:2]

In [ ]:
std_priorities.iloc[:1,1:-1]

## Preprocessing for Hungarian Algorithm (if not considering student project interests)

In [ ]:
std_interests[:3]

In [ ]:
spon_needs[:3]

Extract student interest vectors for hungarian algorithm.

In [ ]:
std_vec = std_interests.iloc[:, 1:].to_numpy()

Extract sponsor needs vectors for hungarian algorithm.

In [ ]:
spon_vec = spon_needs.iloc[:, 1:].to_numpy()

Calculate matrix of Euclidean distances between student interest and sponsor needs vectors.

In [ ]:
distance_matrix = cdist(std_vec, spon_vec)

Store distance matrix into dataframe.

In [ ]:
match_scores = pd.DataFrame(distance_matrix, index=std_interests['netid'], columns=spon_needs['sponsor'])
match_scores.index.name = None
match_scores.columns.name = None
match_scores[:3]

Export csv for analysis.

In [ ]:
match_scores.to_csv('../02_data/std_proj_distances_0116.csv')

### Hungarian (Linear Sum) Algorithm

Match students to the optimal project.

In [ ]:
row_ind, col_ind = linear_sum_assignment(match_scores)

In [ ]:
matches = pd.DataFrame()
matches['student'] = match_scores.index[row_ind]
matches['project'] = match_scores.columns[col_ind]

matches.head()

Add student and project info to matches for analysis.

In [ ]:
matches_info = pd.merge(matches, std_interests, how='inner', left_on='student', right_on='netid')
matches_info = pd.merge(matches_info, std_priorities, how='inner', left_on='netid', right_on='netid')
matches_info = pd.merge(matches_info, spon_needs, how='inner', left_on='project', right_on='sponsor')

matches_info = matches_info.drop(['netid', 'sponsor'], axis=1)
matches_info = matches_info.drop_duplicates().reset_index(drop=True)

In [ ]:
matches_info[:2]

In [ ]:
matches_info.to_csv('../02_data/matches_0116_.csv')

## ALTERNATIVE IF USING PROJECT INTEREST - Hungarian algorithm using multiplier based on project interest

In [ ]:
prty_np[:1]

In [ ]:
prty_np = prty_np.astype(float)
prty_np[prty_np == 2] = 1.5

In [ ]:
prty_np[:1]

In [ ]:
std_interests[:3]

In [ ]:
spon_needs[:3]

Extract student interests vectors.

In [ ]:
std_vec = std_interests.iloc[:, 1:].to_numpy()

Extract sponsor needs vectors.

In [ ]:
spon_vec = spon_needs.iloc[:, 1:].to_numpy()

Calculate matrix of Euclidean distances between student interest and sponsor needs vectors.

In [ ]:
distance_matrix = cdist(std_vec, spon_vec)

In [ ]:
distance_matrix[:1]

Multiply eucledian distances by project interests.

In [ ]:
dist_matrix_multiplier = np.multiply(distance_matrix,prty_np)
dist_matrix_multiplier[:1]

### Hungarian (Linear Sum) Algorithm

In [ ]:
match_scores = pd.DataFrame(dist_matrix_multiplier, index=std_interests['netid'], columns=spon_needs['sponsor'])
match_scores.index.name = None
match_scores.columns.name = None
match_scores[:3]

Exporting csv for analysis.

In [ ]:
match_scores.to_csv('../02_data/05_std_proj_distances_1_15_3multiplier_0116.csv')

In [ ]:
row_ind, col_ind = linear_sum_assignment(match_scores)

In [ ]:
matches = pd.DataFrame()
matches['student'] = match_scores.index[row_ind]
matches['project'] = match_scores.columns[col_ind]

matches.head()

Adding project and student names to matches for analysis.

In [ ]:
matches_info = pd.merge(matches, std_interests, how='inner', left_on='student', right_on='netid')
matches_info = pd.merge(matches_info, std_priorities, how='inner', left_on='netid', right_on='netid')
matches_info = pd.merge(matches_info, spon_needs, how='inner', left_on='project', right_on='sponsor')

matches_info = matches_info.drop(['netid', 'sponsor'], axis=1)
matches_info = matches_info.drop_duplicates().reset_index(drop=True)

In [ ]:
matches_info[:2]

Extracting final matches csv.

In [ ]:
matches_info.to_csv('../02_data/05_matches_1_15_3multiplier_0116.csv')